In [1]:
# Libraries required along the code
import numpy as np
np.set_printoptions(threshold = np.inf)
#from SPI_SLAVE import *

In [2]:
#===========================================================================
# COMPLEMENTARY Function
#===========================================================================
# Function to emulate binary add operation in 8 size vector
def ADD_BIN (vector):
    x = np.uint8(1)
    # Vector is passed to decimal scalar
    decV = np.array([vector[i]*2**(np.size(vector) - i - 1) for i in range(np.size(vector))])
    add = sum(decV) + 1
    # The reverse process is done and passed to binary vector
    binV = np.array([1 if add & (1 << (np.size(vector) - j - 1)) else 0 for j in range(np.size(vector))],dtype = 'uint8')
    
    return binV

In [3]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function describing SPI serial communication protocol for slave peripherals
def SPI_SLAVE (SPI_SLAVE_CLOCK_50,SPI_SLAVE_RESET_InHigh,SPI_SLAVE_SS_InLow,SPI_SLAVE_MOSI_In,
               SPI_SLAVE_SCK_In,SPI_SLAVE_data_In,DATAWIDTH_BUS,STATE_SIZE):
    
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    #//////////// STATES ////////////
    State_IDLE = np.uint8(0)
    State_EDGE = np.uint8(1)
    State_RISE_EDGE = np.uint8(2)
    State_NEW_DATA = np.uint8(3)
    State_FALL_EDGE = np.uint8(4)
    #//////////// SIZES ////////////
    
    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    SPI_SLAVE_MISO_Out = np.uint8(1)
    SPI_SLAVE_newData_Out = np.uint8(0)
    SPI_SLAVE_data_Out = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
    
    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    # Registers (Q)
    global MISO_Register,NewData_Register,DataOut_Register,State_Register,SS_Register
    global MOSI_Register,SCK_Register,SCKOld_Register,Data_Register,BitCounter_Register
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # INPUT LOGIC: Combinational
    # Signals (D)
    MOSI_Signal = SPI_SLAVE_MOSI_In
    SS_Signal = SPI_SLAVE_SS_InLow
    SCK_Signal = SPI_SLAVE_SCK_In
    SCKOld_Signal = SCK_Register
    Data_Signal = Data_Register
    BitCounter_Signal = BitCounter_Register
    
    if State_Register == State_IDLE:
        if SS_Register:
            State_Signal = State_IDLE
            BitCounter_Signal = 0
            Data_Signal = SPI_SLAVE_data_In
        else:
            State_Signal = State_EDGE
            
    elif State_Register == State_EDGE:
        if not SCKOld_Register and SCK_Register:
            State_Signal = State_RISE_EDGE
        elif SCKOld_Register and not SCK_Register:
            State_Signal = State_FALL_EDGE
        else:
            State_Signal = State_EDGE
            
    elif State_Register == State_RISE_EDGE:
        BitCounter_Signal = BitCounter_Register + 1
        Data_Signal = np.array([Data_Register[i] for i in range(0,7)])
        Data_Signal = np.insert(Data_Signal,0,MOSI_Register,axis = 0)
        if BitCounter_Register == DATAWIDTH_BUS-1:
            State_Signal = State_NEW_DATA
        else:
            State_Signal = State_EDGE
    
    elif State_Register == State_NEW_DATA:
        State_Signal = State_EDGE
        BitCounter_Signal = 0
        Data_Signal = SPI_SLAVE_data_In
        
    elif State_Register == State_FALL_EDGE:
        if SS_Register:
            State_Signal = State_IDLE
        else:
            State_Signal = State_EDGE
            
    else:
        State_Signal = State_IDLE
    
    #===========================================================================
    # OUTPUTS
    #===========================================================================
    # OUTPUT LOGIC: Combinational
    # Signals (D)
    MISO_Signal = MISO_Register
    NewData_Signal = NewData_Register
    DataOut_Register = DataOut_Register
    
    if State_Register == State_IDLE:
        MISO_Signal = Data_Register[7]
        NewData_Signal = np.uint8(0)
        DataOut_Signal = DataOut_Register

    elif State_Register == State_EDGE:
        MISO_Signal = MISO_Register
        NewData_Signal = np.uint8(0)
        DataOut_Signal = DataOut_Register
        
    elif State_Register == State_RISE_EDGE:
        MISO_Signal = MISO_Register
        NewData_Signal = np.uint8(0)
        DataOut_Signal = DataOut_Register
    
    elif State_Register == State_NEW_DATA:
        MISO_Signal = MISO_Register
        NewData_Signal = np.uint8(1)
        DataOut_Signal = Data_Register
        
    elif State_Register == State_FALL_EDGE:
        MISO_Signal = Data_Register[7]
        NewData_Signal = np.uint8(0)
        DataOut_Signal = DataOut_Register

    else:
        MISO_Signal = np.uint8(1)
        NewData_Signal = np.uint8(0)
        DataOut_Signal = DataOut_Register
        
    # STATE REGISTER : Sequential
    if SPI_SLAVE_CLOCK_50:
        # Updating global registers
        if SPI_SLAVE_RESET_InHigh:
            MISO_Register = np.uint8(1)
            NewData_Register = np.uint8(0)
            DataOut_Register = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
            State_Register = State_IDLE;
            SS_Register = np.uint8(1)
            MOSI_Register = np.uint8(1)
            SCK_Register = np.uint8(0)
            SCKOld_Register = np.uint8(0)
            Data_Register = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
            BitCounter_Register = 0
        else:
            MISO_Register = MISO_Signal
            NewData_Register = NewData_Signal
            DataOut_Register = DataOut_Signal
            State_Register = State_Signal
            SS_Register = SS_Signal
            MOSI_Register = MOSI_Signal
            SCK_Register = SCK_Signal
            SCKOld_Register = SCKOld_Signal
            Data_Register = Data_Signal
            BitCounter_Register = BitCounter_Signal

    # OUTPUT ASSIGNMENTS
    SPI_SLAVE_MISO_Out = MISO_Register
    SPI_SLAVE_newData_Out = NewData_Register
    SPI_SLAVE_data_Out = DataOut_Register
    
    return SPI_SLAVE_MISO_Out,SPI_SLAVE_newData_Out,SPI_SLAVE_data_Out

In [4]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function in charge of the input and output signals (I/O)
def BB_SYSTEM (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,BB_SYSTEM_SS_InLow,BB_SYSTEM_MOSI_In,
               BB_SYSTEM_SCK_In,BB_SYSTEM_data_In):
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    # Data width of the imput bus
    DATAWIDTH_BUS = 8
    # Size for the states needed into the protocol
    STATE_SIZE = 3

    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    BB_SYSTEM_MISO_Out = np.uint8(1)
    BB_SYSTEM_newData_Out = np.uint8(0)
    BB_SYSTEM_data_Out = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
    
    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # Port map - connection between master ports and signals/registers  
    BB_SYSTEM_MISO_Out,BB_SYSTEM_newData_Out,BB_SYSTEM_data_Out = SPI_SLAVE (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,
                                                                             BB_SYSTEM_SS_InLow,BB_SYSTEM_MOSI_In,
                                                                             BB_SYSTEM_SCK_In,BB_SYSTEM_data_In,
                                                                             DATAWIDTH_BUS,STATE_SIZE)
    
    # OUTPUT ASSIGNMENTS
    return BB_SYSTEM_MISO_Out,BB_SYSTEM_newData_Out,BB_SYSTEM_data_Out

In [5]:
#===========================================================================
# SIMULATION Coding
#===========================================================================
# TB_SYSTEM
# Files to build the test vectors are opened and readed
fMaster = "prueba.txt"
fileMaster = np.fromfile(fMaster,dtype = 'uint8')
fSlave = "prueba.txt"
fileSlave = np.fromfile(fSlave,dtype = 'uint8')

# Parameters to describe the size of the arrays
# CLOCK_RATIO = 1/16th (2^4 = 16 CLOCK cycles) 
CLOCK_RATIO = 4
FREQUENCY = 50e6

# The variables to treat the file are created
nBits = 8
MOSI_In = np.zeros(nBits*np.size(fileMaster)*(2**CLOCK_RATIO),dtype = 'uint8')
data_In = np.zeros((nBits,np.size(fileSlave)),dtype = 'uint8')

# The file with integers array is converted into bits array of type unsigned integers of size 8
t = 0
for r in range(np.size(fileMaster)):
    for s in range(nBits):
        if fileMaster[r] & (1 << s):
            bit = 1
        else:
            bit = 0
        # The vector is given according to the protocol as MSB first
        MOSI_In[t] = bit
        t = t + 1
        
        if fileSlave[r] & (1 << (7 - s)):
            bit1 = 1
        else:
            bit1 = 0
        # The vector is given according to the protocol as MSB first
        data_In[s,r] = bit1

# Variable to emulate the 50MHz clock        
CLOCK_50 = 0

# The test vectors to use for the communication protocol are created
nSim = 1
resetV = np.zeros(nSim,dtype = 'uint8')
resetV[0] = np.uint8(0)
ssV = np.zeros(nSim,dtype = 'uint8')
#ssV[:2] = np.uint8(1)
SCK_In = np.zeros(CLOCK_RATIO,dtype = 'uint8')

# Arrays composed of protocol outputs in each clock cycle
MISO_OutV = np.zeros(nBits*np.size(fileMaster)*(2**CLOCK_RATIO),dtype = 'uint8')
newData_OutV = np.zeros(nBits*np.size(fileSlave)*(2**CLOCK_RATIO),dtype = 'uint8')
data_OutV = np.zeros((nBits,np.size(fileSlave)),dtype = 'uint8')

# The communication protocol is tested for nSim simulations
i = 0
while i < nSim:
    j = 0
    k = 0
    l = 0
    
    # The communication protocol registers are defined on its initial values
    MISO_Register = np.uint8(1)
    NewData_Register = np.uint8(0)
    DataOut_Register = np.zeros(nBits,dtype = 'uint8')
    State_Register = np.uint8(0)
    SS_Register = np.uint8(1)
    MOSI_Register = np.uint8(1)
    SCK_Register = np.uint8(0)
    SCKOld_Register = np.uint8(0)
    Data_Register = np.zeros(nBits,dtype = 'uint8')
    BitCounter_Register = 0
    while j < np.size(data_In):
        CLOCK_50 = np.uint8(0) if CLOCK_50 else np.uint8(1)
        SCK_In = ADD_BIN (SCK_In)
        MISO_Out,newData_Out,data_Out = BB_SYSTEM (CLOCK_50,resetV[i],ssV[i],MOSI_In[j],SCK_In[0],data_In[:,k])
        MISO_OutV[l] = MISO_Out 
        if CLOCK_50:
            newData_OutV[j] = newData_Out 
            if newData_Out:
                data_OutV[:,k] = data_Out
                #print('Out: ',data_OutV[:,k])
                #print('In:  ',data_In[:,k])
                
                k = k + 1
        
        if np.array_equal(SCK_In,np.ones(CLOCK_RATIO,dtype = 'uint8')):
            j = j + 1
        l = l + 1
    i = i + 1

# Out to the receptor MOSI input
np.savetxt('miso_Out.txt',MISO_OutV,fmt = '%u')
print(MISO_OutV)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 